In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, LSTM, Bidirectional
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras import backend as K

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

import sys, os, time, json, datetime, glob

from logzero import logger
import warnings
#import tensorflow as tf
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd
from pathlib import Path

from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()
from art.metrics import RobustnessVerificationTreeModelsCliqueMethod


from art.attacks.evasion import FastGradientMethod, CarliniL0Method
from art.estimators.classification import KerasClassifier

/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/site-packages/art/estimators/certification/__init__.py:28: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


In [2]:
model = Sequential()
model.add(Dense(64, input_dim = 19, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(41, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [3]:
df = pd.read_csv("/rds/general/user/as9820/home/revist/revisiting-iot-device-identification/data/features_nov-apr.csv", parse_dates = ['time_start'])
df.fillna(0, inplace=True)



/var/tmp/pbs.8254068.pbs/ipykernel_3748941/4075129579.py:1: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/rds/general/user/as9820/home/revist/revisiting-iot-device-identification/data/features_nov-apr.csv", parse_dates = ['time_start'])


In [4]:
y = []
packets = [] 
i = 0
    
devs = ['appkettle','blink-security-hub','bosiwo-camera-wifi','lefun-cam-wired','insteon-hub',
            'echoplus','meross-dooropener','smartlife-bulb','xiaomi-ricecooker','ubell-doorbell',
            'appletv','tplink-bulb','google-home','icsee-doorbell','t-wemo-plug','echospot',
            'nest-tstat','sousvide','smartlife-remote','netatmo-weather-station','lgtv-wifi',
            'wansview-cam-wired','xiaomi-plug','xiaomi-hub','lightify-hub','bosiwo-camera-wired',
            'tplink-plug2','allure-speaker','honeywell-thermostat','smarter-coffee-mach','roku-tv',
            'yi-camera','firetv','echodot','smartthings-hub','reolink-cam-wired','t-philips-hub',
            'switchbot-hub','ring-doorbell','blink-camera','samsungtv-wired']

features = ['srcPort', 'destPort',
       'bytes_out', 'num_pkts_out', 'bytes_in', 'num_pkts_in', 'f_ipt_mean',
       'f_ipt_std', 'f_ipt_var', 'f_ipt_skew', 'f_ipt_kurtosis', 'f_b_mean',
       'f_b_std', 'f_b_var', 'f_b_skew', 'f_b_kurtosis', 'duration', 'pr',
       'domainId']
label = 'deviceId'

#weeksToProcess = list(range(44,53)) + list(range(1,19))
weeks = [list(range(44,53)), list(range(1,10)), list(range(10,19))]
weeksToProcess = weeks[0] + weeks[1] + weeks[2]
#weeksToProcess = weeks[0]

In [5]:
trainCon = df['time_start'].dt.isocalendar().week.isin(weeks[0])


In [6]:
testCons = pd.date_range('2019-11-01', '2020-05-01', freq = '1Y').tolist()
lb = LabelBinarizer()
lb.fit(range(1,41))

LabelBinarizer()

In [9]:
model = keras.models.load_model("model.15-0.43.h5", compile=False)
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
classifier = KerasClassifier(model=model, clip_values=(0, 1))

In [12]:
attack_cw = CarliniL0Method(classifier=classifier,
                              max_iter=100,
                              learning_rate=0.01,
                              initial_const=1e0)

for date in testCons:
    testCon = df['time_start'].dt.isocalendar().week == pd.Timestamp(date).week
    #testCon = df['time'].dt.date == pd.Timestamp(date).date
    #testDF = df[testCon]
    testDF = df

    X = testDF[features]
    y = testDF[label]

    if len(X) == 0:
        continue
    
    print("starting scalar")

    scaler = StandardScaler().fit(df[trainCon][features].values)
    X = scaler.transform(X.values)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1200, stratify=y)
            
    labels = lb.transform(y_test)
 
    print("labelled scalar") 
    
    
    x_test_adv = attack_cw.generate(X_test)
    np.savetxt(f"nncw0-{date.week}.csv", x_test_adv, delimiter=",")
    print("cw2 generated")
    loss_test, accuracy_test = model.evaluate(x_test_adv, y_test-1)
    perturbation = np.mean(np.abs((x_test_adv - X_test)))
    print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
    print('Average perturbation: {:4.2f}'.format(perturbation))
    dist=0
    for index, sample in enumerate (x_test_adv):
        dist = dist + np.linalg.norm(sample-X_test[index])
    print(dist/1200)

starting scalar
labelled scalar


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x1520d8e669d0>
Traceback (most recent call last):
  File "/rds/general/user/as9820/home/anaconda3/envs/test1/lib/python3.9/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [15]:
dist=0

model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
#y_test= y_test-1
loss_test, accuracy_test = model.evaluate(x_test_adv, y_test)
perturbation = np.mean(np.abs((x_test_adv - X_test)))
print('Accuracy on adversarial test data: {:4.2f}%'.format(accuracy_test * 100))
for index, sample in enumerate (x_test_adv):
    dist = dist + np.linalg.norm(sample-X_test[index])
print(dist/1200)

Accuracy on adversarial test data: 6.92%
2.585241242687288


In [16]:
dist =  np.linalg.norm(x_test_adv-X_test)
print(dist)

108.73461082864512


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, stratify=y)
X_test = normalize(X=X_test)

rt = RobustnessVerificationTreeModelsCliqueMethod(classifier=classifier)
average_bound, verified_error = rt.verify(x=X_test, y=y_test, eps_init=0.3, nb_search_steps=10, max_clique=2, 
                                          max_level=2)

print('Average bound:', average_bound)
print('Verified error at eps:', verified_error)

AttributeError: 'KerasClassifier' object has no attribute 'get_trees'